# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


class Sample():
    def __init__(cls, name, label, version=None):
        cls.name = name
        cls.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        cls.histo_filename = '../plots1/histos_{}{}.root'.format(cls.name, version)
        cls.histo_file = ROOT.TFile(cls.histo_filename)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext



def drawAll(histograms, 
            labels=None, 
            options='', 
            text=None, 
            norm=False, 
            logy=False, 
            min_y=None, 
            max_y=None, 
            y_axis_label=None,
            do_ratio=False,
            do_profile=False):
    if len(histograms) == 0:
        print 'ERROR: no histogram in input'
        return -1
    if do_ratio and len(histograms) != 2:
        print 'ERROR: do_ratio option only available when 2 histograms need to be drawn!'
        return -2
    if do_profile and not ('TH2' in histograms[0].ClassName() or 'TH3' in histograms[0].ClassName()):
        print 'ERROR: do_profile option only available with TH2 and TH3 histograms'
        return -3
    
    # we build the canvas
    return
    

def drawSame(histograms, 
             labels, 
             options='', 
             norm=False, 
             logy=False, 
             min_y=None, 
             max_y=None, 
             text=None, 
             y_axis_label=None):
    global colors
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2

    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            hist.DrawNormalized('same'+','+options, 1.)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        histograms[0].GetYaxis().SetTitle(y_axis_label)
    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min,max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1,"gaus",minfit,maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")



def drawGraphsSame(histograms, labels, options='', norm=False, logy=False, min_y=None, max_y=None, text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")


In [3]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [4]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v53b'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += '& (gen_sel == @gen_sel)'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
#         print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------
sample_ele_flat2to100_PU0 = Sample('ele_flat2to100_PU0', 'PU0', version)
sample_ele_flat2to100_PU200 = Sample('ele_flat2to100_PU200', 'PU200', version)

samples_ele = [sample_ele_flat2to100_PU0, sample_ele_flat2to100_PU200]

# -------------------------------------------------------------------------

sample_photon_flat8to150_PU0 = Sample('photon_flat8to150_PU0', 'PU0', version)
sample_photon_flat8to150_PU200 = Sample('photon_flat8to150_PU200', 'PU0', version)

samples_photons = [sample_photon_flat8to150_PU0, sample_photon_flat8to150_PU200]


sample_gPt35_PU0 = Sample('photonPt35_PU0', 'Pt35 PU0', version)
sample_gPt35_PU200 = Sample('photonPt35_PU200', 'Pt35 PU200', version)

sample_hadronGun_PU0 = Sample('hadronGun_PU0', 'PU0', version)
sample_hadronGun_PU200 = Sample('hadronGun_PU200', 'PU200', version)


samples_photon = [sample_gPt35_PU0, sample_gPt35_PU200]
samples_hadrons = [sample_hadronGun_PU0, sample_hadronGun_PU200]

sample_nugunrate = Sample('nugun_alleta_pu200', 'PU200', version, version)
samples_nugunrates = [sample_nugunrate]


from python.selections import tp_rate_selections
from python.selections import tp_match_selections
from python.selections import gen_part_selections
from python.selections import eg_qual_selections
from python.selections import eg_rate_selections
tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
# tpset_selections.update(get_label_dict(tp_match_selections))

# gen_selections.update(get_label_dict(gen_part_selections))


gen_part_selections: 15


Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU0_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photonPt35_PU200_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU0_v53b.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_hadronGun_PU200_v53b.root does not exist


In [5]:
tpsets = {'EG': 'EG'}

tpset_selections = {}

gen_selections = {}
samples = []


#tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(eg_rate_selections))
gen_selections.update(get_label_dict(gen_part_selections))
gen_selections.update({'nomatch': 'nomatch'})
samples = samples_ele



In [6]:
tpset_selections


{'EGq1': 'q1',
 'EGq1EtaA': 'q1, |#eta^{L1}| <= 1.52',
 'EGq1EtaAB': 'q1, |#eta^{L1}| <= 1.7',
 'EGq1EtaABC': 'q1, |#eta^{L1}| <= 2.4',
 'EGq1EtaB': 'q1, 1.52 < |#eta^{L1}| <= 1.7',
 'EGq1EtaBC': 'q1, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq1EtaBCD': 'q1, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq1EtaBCDE': 'q1, 1.52 < |#eta^{L1}|',
 'EGq1EtaC': 'q1, 1.7 < |#eta^{L1}| <= 2.4',
 'EGq1EtaD': 'q1, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq1EtaE': 'q1, |#eta^{L1}| > 2.8',
 'EGq2': 'q2',
 'EGq2EtaA': 'q2, |#eta^{L1}| <= 1.52',
 'EGq2EtaAB': 'q2, |#eta^{L1}| <= 1.7',
 'EGq2EtaABC': 'q2, |#eta^{L1}| <= 2.4',
 'EGq2EtaB': 'q2, 1.52 < |#eta^{L1}| <= 1.7',
 'EGq2EtaBC': 'q2, 1.52 < |#eta^{L1}| <= 2.4',
 'EGq2EtaBCD': 'q2, 1.52 < |#eta^{L1}| <= 2.8',
 'EGq2EtaBCDE': 'q2, 1.52 < |#eta^{L1}|',
 'EGq2EtaC': 'q2, 1.7 < |#eta^{L1}| <= 2.4',
 'EGq2EtaD': 'q2, 2.4 < |#eta^{L1}| <= 2.8',
 'EGq2EtaE': 'q2, |#eta^{L1}| > 2.8'}

In [7]:
gen_selections

{'GEN': '',
 'GENEtaA': '|#eta^{GEN}| <= 1.52',
 'GENEtaAB': '|#eta^{GEN}| <= 1.7',
 'GENEtaABC': '|#eta^{GEN}| <= 2.4',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDE': '1.52 < |#eta^{GEN}|',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaE': '|#eta^{GEN}| > 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': 'nomatch'}

In [8]:
samples = samples_nugunrates

tpset_selections = {}
tpset_selections.update(get_label_dict(tp_rate_selections))

tps = ['EG']
tp_select = tpset_selections.keys()


print tps
print tp_select

['EG']
['all', 'EtaAB', 'EmEtaD', 'EmEtaE', 'EmEtaBC', 'EmEtaA', 'EmEtaB', 'EmEtaC', 'EtaD', 'EtaE', 'Emv1EtaC', 'Emv1EtaB', 'Emv1EtaE', 'EtaA', 'EtaABC', 'EtaC', 'Emv1EtaBCDE', 'Emv1EtaBC', 'Emv1EtaD', 'EtaB', 'Emv1', 'EmEtaABC', 'EmEtaBCDE', 'EtaBC', 'Emv1EtaA', 'Emv1EtaBCD', 'Em', 'Emv1EtaAB', 'EmEtaBCD', 'EmEtaAB', 'EtaBCDE', 'Emv1EtaABC', 'EtaBCD']


In [9]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, [])


hplot.cache_histo(classtype=histos.RateHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=None)



<__main__.Sample instance at 0x120bfdc68>
<__main__.Sample instance at 0x120bfdc68> EG all None
<__main__.Sample instance at 0x120bfdc68> EG EtaAB None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaD None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaE None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaBC None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaA None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaB None
<__main__.Sample instance at 0x120bfdc68> EG EmEtaC None
<__main__.Sample instance at 0x120bfdc68> EG EtaD None
<__main__.Sample instance at 0x120bfdc68> EG EtaE None
<__main__.Sample instance at 0x120bfdc68> EG Emv1EtaC None
<__main__.Sample instance at 0x120bfdc68> EG Emv1EtaB None
<__main__.Sample instance at 0x120bfdc68> EG Emv1EtaE None
<__main__.Sample instance at 0x120bfdc68> EG EtaA None
<__main__.Sample instance at 0x120bfdc68> EG EtaABC None
<__main__.Sample instance at 0x120bfdc68> EG EtaC None
<__main__.Sample instance at 0x120bfdc68> EG Emv1Et

In [10]:
hplot.data

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,v53b,PU200,EG,all,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120bba440>
1,v53b,PU200,EG,EtaAB,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120bbaea8>
2,v53b,PU200,EG,EmEtaD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120bbadd0>
3,v53b,PU200,EG,EmEtaE,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120ba7ef0>
4,v53b,PU200,EG,EmEtaBC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c2c0e0>
5,v53b,PU200,EG,EmEtaA,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c2c050>
6,v53b,PU200,EG,EmEtaB,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c2cab8>
7,v53b,PU200,EG,EmEtaC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c1ecb0>
8,v53b,PU200,EG,EtaD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c2c4d0>
9,v53b,PU200,EG,EtaE,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x120c2c200>


In [11]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)


AttributeError: RateHistos instance has no attribute 'h_norm'

#### 3D clusters matched to GEN

In [21]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_norm for his in hsets], labels, text=text)
         

In [23]:
for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_pt for his in hsets], labels, text=text, logy=True)
         

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [25]:
for tp in tps:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, ['EmEtaABC', 'EmEtaBC'], None)
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    hist_list = [his.h_pt for his in hsets]
    drawSame(hist_list, labels, text=text, logy=True)
    hRatio = hist_list[1].Clone(uuid.uuid4().hex[:6])
    stuff.append(hRatio)
    hRatio.Divide(hist_list[0])
    drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [26]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', ['DEF','DEFCalib'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
drawSame(hist_list, labels, text=text, logy=True)
hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
stuff.append(hRatio)
hRatio.Divide(hist_list[1])
drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

### Comparisono baseline vs Manos EGID

In [27]:
for tp in ['DEF']:
    for etaregion in [['EmEtaBC', 'Emv1EtaBC'], ['EmEtaB', 'Emv1EtaB'], ['EmEtaC', 'Emv1EtaC'], ['EmEtaD', 'Emv1EtaD'], ['EmEtaE', 'Emv1EtaE']]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, etaregion, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        hist_list = [his.h_pt for his in hsets]
        drawSame(hist_list, labels, text=text, logy=True)
        hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
        stuff.append(hRatio)
        hRatio.Divide(hist_list[1])
        drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [28]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        # drawSame([his.h_ptVabseta for his in hsets], labels, text=text)
  

In [29]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for his in hsets], labels, text=text+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')



In [30]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for his in hsets], labels, text=text+', p_{T}^{L1}>20GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [31]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for his in hsets], labels, text=text+', p_{T}^{L1}>30GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [32]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for his in hsets], labels, text=text+', p_{T}^{L1}>40GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
